In [2]:
# !pip install tensorflow-gpu

In [3]:
!nvidia-smi

Sat Nov 28 02:13:32 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [60]:
import tensorflow as tf

In [88]:
# import libraries
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
from imutils import paths
import matplotlib.pyplot as plt
%matplotlib inline

In [62]:
# resize all the images
IMAGE_SIZE = [224, 224]

In [63]:
train_path = '/content/drive/MyDrive/cdata/train' 
valid_path = '/content/drive/MyDrive/cdata/test'

In [75]:
# Import the Resnet50 library as shown below and add preprocessing layer
# Here we will be using imagenet weights

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [76]:
# don't train existing wights
for layer in vgg.layers:
    layer.trainble = False

In [77]:
folder = glob('/content/drive/MyDrive/cdata/train/*')
folder

['/content/drive/MyDrive/cdata/train/Alice Englert_train',
 '/content/drive/MyDrive/cdata/train/Cynthia Nixon_train',
 '/content/drive/MyDrive/cdata/train/Finn Wittrock_train',
 '/content/drive/MyDrive/cdata/train/Jon Jon Briones_train',
 '/content/drive/MyDrive/cdata/train/Judy Davis_train',
 '/content/drive/MyDrive/cdata/train/Sarah Paulson_train']

In [78]:
#Flatten because can able to add last number of layers
x = Flatten()(vgg.output)

In [79]:
prediction = Dense(len(folder),activation='softmax')(x)
model = Model(inputs=vgg.input,outputs = prediction)# model object

In [80]:
# view the summary
model.summary()

Model: "functional_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)     

In [81]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [82]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [83]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/cdata/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 228 images belonging to 6 classes.


In [84]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/cdata/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 76 images belonging to 6 classes.


In [86]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/50
8/8 [==============================] - 20s 2s/step - loss: 0.7310 - accuracy: 0.7149 - val_loss: 1.3760 - val_accuracy: 0.6316
Epoch 2/50
8/8 [==============================] - 19s 2s/step - loss: 0.6514 - accuracy: 0.7719 - val_loss: 1.3715 - val_accuracy: 0.6184
Epoch 3/50
8/8 [==============================] - 19s 2s/step - loss: 0.6103 - accuracy: 0.7500 - val_loss: 1.8457 - val_accuracy: 0.6053
Epoch 4/50
8/8 [==============================] - 19s 2s/step - loss: 0.5632 - accuracy: 0.7807 - val_loss: 1.5028 - val_accuracy: 0.6974
Epoch 5/50
8/8 [==============================] - 19s 2s/step - loss: 0.7101 - accuracy: 0.7325 - val_loss: 1.6928 - val_accuracy: 0.5789
Epoch 6/50
8/8 [==============================] - 19s 2s/step - loss: 0.9451 - accuracy: 0.6667 - val_loss: 1.5762 - val_accuracy: 0.6184
Epoch 7/50
8/8 [==============================] - 21s 3s/step - loss: 0.9498 - accuracy: 0.7018 - val_loss: 1.6098 - val_accuracy: 0.6842
Epoch 8/50
8/8 [==================

In [90]:
from keras.models import load_model

model.save('vgg16_new_model.h5')